<a href="https://colab.research.google.com/github/ashutosh3060/Agentic-RAG/blob/main/Overview_of_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Purpose of the notebook: This notebook is meant for basic RAG set up

## Table of Contents
### 0. Pre-requisites
### 1. Overview of all the components of RAG
### 2. Indexing
### 3. Retrieval
### 4. Generation

### 0. Pre-requisites

Option 1: Install all the dependent libraries inside a virtual environment

* Install virtual environment library
* Create a virtual environment
* Activate the environment
* Install the required libraries from requirements.txt file

Option 2: Install dependent libraries onn the global python environment


#### Option 1: Install all the dependent libraries inside a virtual environment

In [1]:
!pip install virtualenv
!virtualenv "rag_venv" # To set up the env

!source /content/rag_venv/bin/activate # To activate the environment and download all its dependencies and packages


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 29.6 MB/s eta 0:00:00
created virtual environment CPython3.11.11.final.0-64 in 925ms
  creator CPython3Posix(dest=/content/rag_venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.0.1, setuptools==75.8.0, wheel==0.45.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [2]:
# !source /content/myenv/bin/activate; pip3 list

In [3]:
# ! which python

In [7]:
# Install all packages
! pip install -r /content/requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.7 MB/s eta 0:0

#### Option 2: Install dependent libraries onn the global python environment

In [8]:
# ! pip install --quiet pinecone-client python-dotenv langchain langchain-community langchain-core langchain-openai beautifulsoup4 tiktoken numpy


### Environment

(1) Packages

In [12]:
import os
from dotenv import load_dotenv

# Load all environment variables from .env file
load_dotenv()

# Access the environment variables (Langchain)
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_endpoint = os.getenv('LANGCHAIN_ENDPOINT')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')

# LLM
openai_api_key = os.getenv('OPENAI_API_KEY')

# Pinecone Vector Database
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pinecone_api_host = os.getenv('PINECONE_API_HOST')
index_name = os.getenv('PINECONE_INDEX_NAME')

In [13]:
os.environ['LANGCHAIN_TRACING_V2'] = langchain_tracing_v2
os.environ['LANGCHAIN_ENDPOINT'] = langchain_endpoint
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key

In [14]:
os.environ['OPENAI_API_KEY'] = openai_api_key

#Pinecone keys
os.environ['PINECONE_API_KEY'] = pinecone_api_key
os.environ['PINECONE_API_HOST'] = pinecone_api_host
os.environ['PINECONE_INDEX_NAME'] = index_name

Pinecone Init

In [15]:
from pinecone import Pinecone

pc = Pinecone(api_key = os.environ['PINECONE_API_KEY'])
index = pc.Index(os.environ['PINECONE_INDEX_NAME'])

### 1. Overview of all the components of RAG

In [17]:
# Import all the libraries

#Beautiful Soup is a library that makes it easy to scrape information from web pages
import bs4

# Langchain Libraries
# Taking inspiration from Hugging Face Hub, LangChainHub is collection of all artifacts useful for working with LangChain primitives such as prompts, chains and agents
from langchain import hub
# take a document and split into chunks that can be used for retrieval
from langchain.text_splitter import RecursiveCharacterTextSplitter
# loading documents from a variety of source
from langchain_community.document_loaders import WebBaseLoader
# Working with Pinecone vector store
from langchain_community.vectorstores import Pinecone
# Langchain output parser
from langchain_core.output_parsers import StrOutputParser
# to passthrough inputs unchanged or with additional keys
from langchain_core.runnables import RunnablePassthrough
# OpenAI model and embeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


* Indexing

In [23]:
# Load Documents

loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs = dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
            )
        )
)

docs = loader.load()

In [9]:
from pprint import pprint
import bs4 --Beautiful Soup is a library that makes it easy to scrape information from web pages

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Pinecone.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
    index_name=index_name
)

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
pprint(rag_chain.invoke("How does LangChain use vector stores for efficient data retrieval?"))